- get rain data and visualise for overview <font color='orange'>&#9728;</font>
- cut and split data to reasonable train, val, test sets <font color='orange'>&#9728;</font>
- set up first generator and discriminator and overfit, play around with it, get first results (_November/December_)
- train on small training set and improve losses, accuracy, results (_December_)
- experiment with temporal discriminator (_December/January_)
- experiment with wind data (_January_)

## Imports and settings

In [ ]:
import preproc

In [ ]:
from os import *
import numpy as np
import sys
import re
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')

In [ ]:
#forces CPU usage
environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
environ["CUDA_VISIBLE_DEVICES"] = "0" #"" for CPU
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

______________________________________-
## Data preprocessing

### Rain measurements
Measurements are downloaded from the DWD (German weather service) open data server: __ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/__<br>
I'm working with the data of August 2010 (based on [this](https://tradingeconomics.com/germany/precipitation)), so I have downloaded this: __ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/radolan/historical/asc/2010/RW-201008.tar__<br>
_DWD manual:_<br>
__ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/radolan/historical/asc/BESCHREIBUNG_gridsgermany_hourly_radolan_historical_asc_de.pdf__<br><br>
This contains radar maps recorded in every hour. Each map has a resolution of $900\times900$ pixels and each pixel corresponds to an $1\,km\times1\,km$ area in reality. Pixel values are the precipitation height in $0.1\,mm$.
Below I'm importing the data of this as a series of numpy arrays and plot them to see the acual radar map. The _sys.path[0]_ is the path on my computer and it can be different for you.

In [ ]:
dataDir = sys.path[0]+"/rain"
files = listdir(dataDir)
files[0:5]

In [ ]:
totalLength = len(files)
#array of inputs
inputs = np.empty((totalLength,900,900))
print(inputs.shape)

In [ ]:
#import data to the inputs array
for i, file in enumerate(files):
    clear_output(wait=True)
    print(f"[{i+1}/{totalLength}]")
    ascii_grid = np.loadtxt(f"{dataDir}/{files[i]}", skiprows=6)
    inputs[i] = ascii_grid

In [ ]:
inputs = preproc.mask_data(inputs,100)

In [ ]:
# look over the data
%matplotlib notebook
from  matplotlib.animation import FuncAnimation

fig = plt.figure()
plot = plt.imshow(np.ma.masked_where(inputs[0] < 0, inputs[0])) #inputs 100 has a good mask
plt.colorbar()
plt.xlabel("Longitude pixels")
plt.ylabel("Latitude pixels")
def update(i):
    plot.set_data(np.ma.masked_where(inputs[i] < 0, inputs[i]))
    plt.title(f"Index in array: {i}\nDate: {files[i][9:11]}.{files[i][7:9]}.{files[i][3:7]} {files[i][12:14]}:{files[i][14:16]}")
    return plot

anim = FuncAnimation(fig, update, frames=totalLength, interval = 200, blit=True, repeat=False)

plt.show()

In [ ]:
train, xval, test = preproc.generate_datasets(inputs, n=100, size=64, length=3, split=(0.6,0.2,0.2))

In [ ]:
%matplotlib inline
visualise_data(train)

# Architecture
This is a toy network.

In [ ]:
D = keras.Sequential()
depth = 2
dropout = 0.4

conv1 = keras.layers.Conv2D(filters=depth*1 ,kernel_size=10, strides=5, input_shape=(900,900,1), padding='same')
relu1 = keras.layers.LeakyReLU(alpha=0.2)
dropout1 = keras.layers.Dropout(dropout)

conv2 = keras.layers.Conv2D(filters=depth*2, kernel_size=10, strides=5, padding='same')
relu2 = keras.layers.LeakyReLU(alpha=0.2)
dropout2 = keras.layers.Dropout(dropout)

conv3 = keras.layers.Conv2D(filters=depth*4, kernel_size=6, strides=2, padding='same')
relu3 = keras.layers.LeakyReLU(alpha=0.2)
dropout3 = keras.layers.Dropout(dropout)

conv4 = keras.layers.Conv2D(filters=depth*8, kernel_size=2, strides=2, padding='same')
relu4 = keras.layers.LeakyReLU(alpha=0.2)
dropout4 = keras.layers.Dropout(dropout)

# Out: 1-dim probability
flatten = keras.layers.Flatten()
fcl1 = keras.layers.Dense(1)
sig1 = keras.layers.Activation('sigmoid')
#summary()
D.add(conv1)
D.add(relu1)
D.add(dropout1)
D.add(conv2)
D.add(relu2)
D.add(dropout2)
D.add(conv3)
D.add(relu3)
D.add(dropout3)
D.add(conv4)
D.add(relu4)
D.add(dropout4)
D.add(flatten)
D.add(fcl1)
D.add(sig1)
D.summary()

In [ ]:
G = keras.Sequential()
dropout = 0.4
depth = 32
dim = 9
        # In: 100
        # Out: dim x dim x depth
G.add(keras.layers.Dense(dim*dim*depth, input_dim=100))
G.add(keras.layers.BatchNormalization(momentum=0.9))
G.add(keras.layers.Activation('relu'))
G.add(keras.layers.Reshape((dim, dim, depth)))
G.add(keras.layers.Dropout(dropout))

G.add(keras.layers.UpSampling2D(size=(2, 2)))
G.add(keras.layers.Conv2DTranspose(int(depth/2), 5, padding='same'))
G.add(keras.layers.BatchNormalization(momentum=0.9))
G.add(keras.layers.Activation('relu'))

G.add(keras.layers.UpSampling2D(size=(2, 2)))
G.add(keras.layers.Conv2DTranspose(int(depth/4), 5, padding='same'))
G.add(keras.layers.BatchNormalization(momentum=0.9))
G.add(keras.layers.Activation('relu'))

G.add(keras.layers.UpSampling2D(size=(5, 5)))
G.add(keras.layers.Conv2DTranspose(int(depth/8), 5, padding='same'))
G.add(keras.layers.BatchNormalization(momentum=0.9))
G.add(keras.layers.Activation('relu'))

G.add(keras.layers.UpSampling2D(size=(5, 5)))
G.add(keras.layers.Conv2DTranspose(1, 5, padding='same'))
G.add(keras.layers.Activation('sigmoid'))
G.summary()

In [ ]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001, decay=3e-8)
AM = keras.Sequential()
AM.add(G)
AM.add(D)
AM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
AM.fit(train_val_inputs,
          train_val_targets,
          batch_size = 50,
          epochs=50,
          validation_split = 0.1,
          shuffle = True)